# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#load csv
file = os.path.join("..","WeatherPy","weather_df.csv")
weather_df = pd.read_csv(file)

#set index 
weather_df = weather_df.drop("Unnamed: 0", axis = 1)
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,50.00,87,75,8.05,CL,1619113847
1,Busselton,-33.6500,115.3333,57.99,79,100,1.01,AU,1619113847
2,Bluff,-46.6000,168.3333,48.00,96,94,3.00,NZ,1619113847
3,Aksarka,66.5606,67.7975,36.90,90,100,16.22,RU,1619113847
4,Castro,-24.7911,-50.0119,63.70,71,85,1.79,BR,1619113848


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
figure_layout = {'width': '800px','height': '500px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

#convert lats lngs to list
lats = weather_df["Lat"].tolist()
lngs = weather_df["Lng"].tolist()

a_zip = zip(lats,lngs)
coordinates = list(a_zip)

#print(coordinates)

markers = gmaps.marker_layer(coordinates)
weights = weather_df["Humidity"]

fig.add_layer(gmaps.heatmap_layer(coordinates,weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
great_weather_df = weather_df.loc[(weather_df["Max Temp"] < 75) & (weather_df["Humidity"] < 70) & (weather_df["Cloudiness"] <50)]
great_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Broken Hill,-31.9500,141.4333,50.00,66,0,8.05,AU,1619113849
13,Alice Springs,-23.7000,133.8833,48.20,61,0,2.30,AU,1619113805
32,Chui,-33.6971,-53.4616,73.65,54,13,13.44,UY,1619113854
34,Fochville,-26.4886,27.4939,63.00,63,39,0.83,ZA,1619113855
37,Serebryansk,49.6925,83.2892,30.20,64,20,13.42,KZ,1619113856
...,...,...,...,...,...,...,...,...,...
523,Fairbanks,64.8378,-147.7164,45.00,44,1,6.91,US,1619113777
528,Belyy Yar,53.6039,91.3903,30.20,51,0,8.95,RU,1619114004
532,Brisbane,-27.4679,153.0281,57.99,62,0,5.75,AU,1619114005
539,Plainview,40.7765,-73.4673,48.00,31,40,20.71,US,1619114007


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()
hotel_df["Name"]=""
hotel_df["Lat"]=""
hotel_df["Lng"]=""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location":"-31.9500,141.4333",
    "radius": 5000,
    "type":"Hotel",
    "key":g_key
}


#print(response.url)
for row in great_weather_df.iterrows():
    latlng = f"{row[1]['Lat']},{row[1]['Lng']}"
    params["location"] = latlng
    response = requests.get(base_url,params=params)
    data = response.json()
    results=data["results"]
    #pprint(results)
    try:
        print(f"The closest hotel to {latlng} is {results[1]['name']} at {results[1]['geometry']['location']['lat']},{results[1]['geometry']['location']['lng']}")
        name = results[1]['name']
        lat = results[1]['geometry']['location']['lat']
        lng = results[1]['geometry']['location']['lng']
        hotel_df = hotel_df.append({"Name":name, "Lat":lat,"Lng":lng}, ignore_index = True)


        
    except (KeyError,IndexError):
        print(f"Couldn't find hotel. No results.")
    
              


The closest hotel to -31.95,141.4333 is Royal Exchange Hotel at -31.95777949999999,141.4657271
The closest hotel to -23.7,133.8833 is Desert Palms Alice Springs at -23.7133953,133.8798204
The closest hotel to -33.6971,-53.4616 is Nuevo Hotel Plaza at -33.6941772,-53.45563299999999
The closest hotel to -26.4886,27.4939 is Engen Fochville (Philip Motors) at -26.4914399,27.495126
The closest hotel to 49.6925,83.2892 is Halyk Bank at 49.6814482,83.2969193
The closest hotel to 54.5167,9.55 is Hotel Hohenzollern at 54.5213273,9.5566289
The closest hotel to 47.7333,-3.4333 is Best Western Plus Hotel Les Rives Du Ter at 47.7291381,-3.3873468
The closest hotel to -37.6167,-73.65 is Restaurante Casa Vieja at -37.60719479999999,-73.6543489
The closest hotel to 37.1041,-113.5841 is Best Western Plus Abbey Inn at 37.0890245,-113.5842669
The closest hotel to 45.0205,-93.21799999999999 is Holiday Inn Express Roseville-St. Paul at 45.02267,-93.193242
The closest hotel to -42.7826,147.0587 is The Shing

The closest hotel to 29.5667,-104.4167 is Movistar at 29.5590086,-104.4137817
The closest hotel to 64.8378,-147.7164 is Westmark Fairbanks Hotel & Conference Center at 64.8397457,-147.712353
The closest hotel to 53.6039,91.3903 is Administratsiya Altayskogo Rayona at 53.600095,91.378984
The closest hotel to -27.4679,153.0281 is Royal On The Park Brisbane at -27.472858,153.029187
The closest hotel to 40.7765,-73.4673 is Four Points by Sheraton Melville Long Island at 40.7848291,-73.44652409999999
The closest hotel to 46.8436,-1.8749 is Citotel Le Challans at 46.844121,-1.8723491


In [7]:
hotel_df
    

,Name,Lat,Lng
0,Royal Exchange Hotel,-31.957779,141.465727
1,Desert Palms Alice Springs,-23.713395,133.879820
2,Nuevo Hotel Plaza,-33.694177,-53.455633
3,Engen Fochville (Philip Motors),-26.491440,27.495126
4,Halyk Bank,49.681448,83.296919
...,...,...,...
89,Westmark Fairbanks Hotel & Conference Center,64.839746,-147.712353
90,Administratsiya Altayskogo Rayona,53.600095,91.378984
91,Royal On The Park Brisbane,-27.472858,153.029187
92,Four Points by Sheraton Melville Long Island,40.784829,-73.446524


In [8]:
##### I didn't know what this code did but I didn't have to run it to get my last figure to work?############


# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [9]:
#same figure as previously shown but I have narrowed it down to my filtered "great weather" df
gmaps.configure(api_key=g_key)
figure_layout = {'width': '800px','height': '500px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

#convert lats lngs to list
lats = great_weather_df["Lat"].tolist()
lngs = great_weather_df["Lng"].tolist()

a_zip = zip(lats,lngs)
coordinates = list(a_zip)

#print(coordinates)

markers = gmaps.marker_layer(coordinates)
weights = great_weather_df["Humidity"]

fig.add_layer(gmaps.heatmap_layer(coordinates,weights))


# Add marker layer ontop of heat map
hotel_lats = hotel_df["Lat"].tolist()
hotel_lngs = hotel_df["Lng"].tolist()

a_zip = zip(hotel_lats,hotel_lngs)
hotel_coordinates = list(a_zip)

hotel_markers = gmaps.marker_layer(hotel_coordinates)
fig.add_layer(hotel_markers)
fig


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…